In [74]:
import os
from checker import *
from entities import *
from utils import *

# !mvn clean package

Análisis de instancias de datasets "a", "b" y "x"

In [75]:
datasets = {"a": {}, "b": {}, "x": {}}
instances = {}

In [76]:
%%capture

for dataset in ["a", "b", "x"]:
    for file in os.listdir(f"datasets/{dataset}"):
        instanceId = file.split("_")[1].split(".")[0]
        instance = Instance(dataset, instanceId, f"datasets/{dataset}/{file}")
        instances[f"{dataset}/{instanceId}"] = instance
        datasets[dataset][instanceId] = instance

In [77]:
# display_stats_table(datasets["a"].values())

In [78]:

# display_stats_table(datasets["b"].values())

In [79]:
# display_stats_table(datasets["x"].values())

Selección de instancias para experimentos

In [80]:
experiment_instances = {
    "a": [i for i in datasets["a"].values() if i.id in ["0001", "0004", "0009", "0017", "0019"]],
    "b": [i for i in datasets["b"].values() if i.id in ["0001", "0003", "0005", "0007", "0009"]],
    "x": [i for i in datasets["x"].values() if i.id in ["0001", "0003", "0006", "0007", "0008"]]
}

In [81]:
# display_stats_table(experiment_instances["a"])

In [82]:
# display_stats_table(experiment_instances["b"])

In [83]:
# display_stats_table(experiment_instances["x"])

Ejecutar con algoritmo greedy

In [84]:

# for dataset in ["a", "b", "x"]:
#     for instance in experiment_instances[dataset]:
        
#         print(f"Processing instance: {dataset}/{instance.id}")
#         !java -jar target/ChallengeSBPO2025-1.0.jar datasets/{dataset}/instance_{instance.id}.txt greedy
#         print("\n")


Verificación de factibilidad y valores objetivo

In [85]:
# %%capture
# checker = WaveOrderPicking()

# algorithm = "greedy"

# for dataset in ["a", "b", "x"]:
#     for instance in experiment_instances[dataset]:
#         result = checker.check_result(f"datasets/{dataset}/instance_{instance.id}.txt", f"output/{algorithm}/{dataset}/instance_{instance.id}.txt")
#         print(f"Result for instance {dataset}/instance_{instance.id}.txt: {result}")

Mejores soluciones publicadas

In [86]:
# for dataset in ["a", "b", "x"]:
#     for instance in experiment_instances[dataset]:
#         result = checker.check_result(f"datasets/{dataset}/instance_{instance.id}.txt", f"best_solutions/{dataset}/instance_{instance.id}.txt")
#         print(f"Result for instance {dataset}/instance_{instance.id}.txt: {result}")
#         if result["is_feasible"]:
#             instance.best_result = result["objective_value"]

## Experimentos

In [87]:
from entities import Experiment

experiments = {}
Experiment.default_parameters = {
        "generations" : 50,
        "population_size" : 60,
        "crossover_rate" : 0.9,
        "mutation_rate" : 0.001,
        "initial_seed" : 12345,
        "iterations" : 1
    }
independent_runs = 10

### 1. Configuración de algoritmos genéticos

In [88]:
algorithms1 = ["ssGA", "gGA"]
batch1 = "algorithm_configuration"
dataset1 = "a"

#### 1.1. Codificación de soluciones: cadenas binarias vs subconjuntos

In [94]:
batch_name = os.path.join(batch1, "encoding")
encodings = ["subset", "binary"]
experiments[batch_name] = []


for instance in experiment_instances[dataset1]:
    for algorithm in algorithms1:
        for encoding in encodings:
            for run in range(independent_runs):
                exp = Experiment(batch_name, instance, algorithm, run)
                exp.set_param("encoding", encoding)
                experiments[batch_name].append(exp)

for exp in experiments[batch_name]:
    exp.run() # execute the experiment or get results from results/... if available


#### 1.2. Operador de cruzamiento: HUX vs unión de órdenes

In [96]:
batch_name = os.path.join(batch1, "crossover")
cx_types = ["default", "orders_union"]
experiments[batch_name] = []

for instance in experiment_instances[dataset1]:
    for algorithm in algorithms1:
        for cx_type in cx_types:
            for run in range(independent_runs):
                exp = Experiment(batch_name, instance, algorithm, run)
                exp.set_param("crossover_type", cx_type)
                experiments[batch_name].append(exp)

for exp in experiments[batch_name]:
    exp.run() # execute the experiment or get results from results/... if available

#### 1.3. Inicialización: aleatoria o warm start

In [103]:
batch_name = os.path.join(batch1, "start")
starts = ["random", "warm"]
experiments[batch_name] = []
run = 1
max_generations = 100
step = 5

for instance in [i for i in experiment_instances[dataset1] if i.id in ["0004", "0017"]]:
    for algorithm in algorithms1:
        for start in starts:
            for generations in range(5, max_generations+1, step):
                exp = Experiment(batch_name, instance, algorithm, run)
                exp.set_param("start", start)
                exp.set_param("generations", generations)
                experiments[batch_name].append(exp)

for exp in experiments[batch_name]:
    exp.run() # execute the experiment or get results from results/... if available

### 2. Configuración de parámetros

In [92]:
batch2 = "parameter_tuning"

### 3. Evaluación


In [93]:
batch3 = "evaluation"

<!-- ## Análisis comparativo -->